# Демонстрация использования класса `FileOrganizer`

Этот ноутбук показывает, как использовать класс `FileOrganizer` из `filesystem.organizer` для выполнения различных операций по организации файлов осциллограмм:
- Обновление времени модификации файлов.
- Извлечение информации о частоте сети и дискретизации из `.cfg` файлов.
- Группировка файлов осциллограмм по этим частотам.
- Создание инвентарного CSV-файла с хеш-суммами `.dat` файлов.

In [ ]:
import os
import shutil
import datetime
import pandas as pd # For displaying CSV content
import numpy as np # For creating sample DAT content
import sys

# Настройка путей и sys.path для импорта модулей проекта
module_path = os.path.abspath(os.path.join(os.getcwd(), '..')) 
if module_path not in sys.path:
    sys.path.append(module_path)

from filesystem.organizer import FileOrganizer

# Создаем временную директорию для демонстрации
base_sample_dir = "temp_sample_data_organizer"
source_dir = os.path.join(base_sample_dir, "source_oscillograms")

if os.path.exists(base_sample_dir):
    shutil.rmtree(base_sample_dir)
os.makedirs(source_dir)

# Функция для создания демонстрационных CFG/DAT файлов
def create_sample_comtrade(path, name, station, device, rev, analog_channels, digital_channels, freq, samp_rate, samp_count, start_ts_str, trigger_ts_str, dat_lines_count=3):
    cfg_content = f"{station},{device},{rev}\n"
    cfg_content += f"{len(analog_channels) + len(digital_channels)},{len(analog_channels)}A,{len(digital_channels)}D\n"
    for idx, ch_name in enumerate(analog_channels):
        cfg_content += f"{idx+1},{ch_name},A,,V,1.0,0,0,0,100,1,1,P\n"
    for idx, ch_name in enumerate(digital_channels):
        cfg_content += f"{idx+1+len(analog_channels)},{ch_name},,,1\n"
    cfg_content += f"{freq}\n"
    cfg_content += "1\n" # nrates = 1
    cfg_content += f"{samp_rate},{samp_count}\n"
    cfg_content += f"{start_ts_str}\n"
    cfg_content += f"{trigger_ts_str}\n"
    cfg_content += "ASCII\n1.0\n"

    with open(os.path.join(path, f"{name}.cfg"), "w", encoding="utf-8") as f:
        f.write(cfg_content)
    
    dat_content = ""
    for i in range(dat_lines_count):
        dat_content += f"{i+1},{i*1000}" # timestamp in us
        for _ in analog_channels: dat_content += f",{np.random.randint(0,100)}"
        for _ in digital_channels: dat_content += f",{np.random.randint(0,1)}"
        dat_content += "\n"
    with open(os.path.join(path, f"{name}.dat"), "w", encoding="utf-8") as f:
        f.write(dat_content)

# Создаем файлы
create_sample_comtrade(source_dir, "osc1_50Hz_1kHz", "StationA", "Dev1", "1999", ["Ua","Ia"], ["Status1"], 50.0, 1000.0, 1000, "01/01/2023,00:00:00.000000", "01/01/2023,00:00:01.000000")
create_sample_comtrade(source_dir, "osc2_60Hz_2kHz", "StationB", "Dev2", "2013", ["Ub","Ib"], ["Status2"], 60.0, 2000.0, 2000, "02/02/2023,00:00:00.000000", "02/02/2023,00:00:01.000000")
create_sample_comtrade(source_dir, "osc3_50Hz_1kHz_alt", "StationC", "Dev3", "1999", ["Uc"], [], 50.0, 1000.0, 500, "03/03/2023,00:00:00.000000", "03/03/2023,00:00:01.000000")
create_sample_comtrade(source_dir, "osc4_malformed_freq", "StationD", "Dev4", "1999", ["I_misc"], [], "invalid", "invalid", 300, "04/04/2023,00:00:00.000000", "04/04/2023,00:00:01.000000") # Malformed freq line
# Create a file that will not be moved (no .dat)
with open(os.path.join(source_dir, "osc5_no_dat.cfg"), "w", encoding="utf-8") as f: f.write("dummy_cfg_content_no_dat")


print(f"Создана демонстрационная директория: {source_dir}")
print("Содержимое директории:")
for item in os.listdir(source_dir):
    print(f"- {item}")

## 1. Инициализация `FileOrganizer`

In [ ]:
organizer = FileOrganizer()
# Для более подробного вывода можно передать is_print_message=True в методы ниже

## 2. Обновление времени модификации файлов (`update_modification_times`)

In [ ]:
import time

print("Время модификации до обновления:")
for item in os.listdir(source_dir):
    stat_info = os.stat(os.path.join(source_dir, item))
    print(f"- {item}: {datetime.datetime.fromtimestamp(stat_info.st_mtime)}")

print("\nОбновляем время модификации...")
# Установим конкретное время для наглядности
custom_datetime = datetime.datetime(2020, 1, 1, 12, 0, 0)
organizer.update_modification_times(source_dir, new_mod_time=custom_datetime)

print("\nВремя модификации после обновления:")
for item in os.listdir(source_dir):
    stat_info = os.stat(os.path.join(source_dir, item))
    print(f"- {item}: {datetime.datetime.fromtimestamp(stat_info.st_mtime)}")

## 3. Извлечение частот из `.cfg` файла (`extract_frequencies_from_cfg`)

In [ ]:
cfg_path1 = os.path.join(source_dir, "osc1_50Hz_1kHz.cfg")
cfg_path_malformed = os.path.join(source_dir, "osc4_malformed_freq.cfg")

print(f"Извлечение частот из {os.path.basename(cfg_path1)}:")
freq_net1, freq_rate1 = organizer.extract_frequencies_from_cfg(cfg_path1, encoding='utf-8', threshold=0.1)
print(f"  Частота сети: {freq_net1} Гц, Частота дискретизации: {freq_rate1} Гц")

print(f"\nИзвлечение частот из {os.path.basename(cfg_path_malformed)} (ожидаются ошибки/None):")
freq_net_mal, freq_rate_mal = organizer.extract_frequencies_from_cfg(cfg_path_malformed, encoding='utf-8')
print(f"  Частота сети: {freq_net_mal}, Частота дискретизации: {freq_rate_mal}")

## 4. Группировка файлов по частотам (`group_files_by_frequency_and_rate`)

Этот метод сканирует указанную директорию, извлекает частоты из `.cfg` файлов и перемещает соответствующие пары `.cfg` и `.dat` в поддиректории вида `f_network = X and f_rate = Y`.

In [ ]:
print(f"\nЗапуск группировки файлов в директории: {source_dir}")
organizer.group_files_by_frequency_and_rate(source_dir, is_print_message=True)

print("\nСтруктура директории после группировки:")
for root, dirs, files in os.walk(source_dir):
    level = root.replace(source_dir, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f"{indent}{os.path.basename(root)}/")
    sub_indent = ' ' * 4 * (level + 1)
    for f in files:
        print(f"{sub_indent}{f}")

print("\nПроверка оставшихся файлов в корневой директории источника:")
for item in os.listdir(source_dir):
    if os.path.isfile(os.path.join(source_dir, item)):
        print(f"- {item}")

## 5. Создание инвентарного CSV с хешами `.dat` файлов (`generate_dat_hash_inventory`)

Этот метод работает с директорией, уже структурированной по частотам (как после `group_files_by_frequency_and_rate`). Он создает CSV-файл с информацией о каждом `.dat` файле, включая его хеш-сумму.

In [ ]:
inventory_csv_path = os.path.join(base_sample_dir, "dat_inventory.csv")

print(f"\nСоздание инвентарного CSV для директории: {source_dir}")
organizer.generate_dat_hash_inventory(source_dir, inventory_csv_path)

if os.path.exists(inventory_csv_path):
    print(f"\nСодержимое инвентарного CSV ({inventory_csv_path}):")
    try:
        inventory_df = pd.read_csv(inventory_csv_path)
        if pd.__version__ > '1.0.0':
            try:
                from IPython.display import display
                display(inventory_df)
            except ImportError:
                print(inventory_df)
        else:
            print(inventory_df)
    except Exception as e:
        print(f"Ошибка чтения CSV: {e}")
else:
    print(f"Инвентарный CSV не был создан (возможно, не было подходящих поддиректорий или .dat файлов).")

In [ ]:
# Очистка временной директории
try:
    if os.path.exists(base_sample_dir):
        shutil.rmtree(base_sample_dir)
        print(f"\nВременная директория {base_sample_dir} удалена.")
except Exception as e:
    print(f"Ошибка при удалении временной директории {base_sample_dir}: {e}")